## Suicide prevention using causality in machine learning

The alarming increase in the number of people affected by mental illness has become one of the main public health problems facing governments around the world.

Suicide is among the 20 most common causes of death according to an assessment by the World Health Organization (Organization et al., 2019), surpassing malaria, breast cancer or homicide and in the last 20 years, there has been a 24% increase in cases.

We used clinical classification data and identified possible causal factors that could establish relationships with suicidal ideation.

To this end, we use causal inference and machine learning methods.
The factors used for causal inference were: Sex, Marital Status, Type of Residence, Alcoholic, Drugs, Suicide in the Family, Depression in the Family, Alcoholic in the Family, Drugs in the Family, Neuro, Psychiatric, Capacity of enjoying things, Impact of your Family and friends, Capacity of making important decisions, Student, Insomnia, Anxiety, Loss of insights, Appetite, Weight loss, Somatic anxiety, Hypochondriasis, Feeling of guilt, Work and interests, Energy, Slow thought and speech, Agitation, Libido, OCD and age.
After our research, the factors that emerged as possible causes that lead an individual to have suicidal ideation were: Suicide in the family, Drugs in the family, Capacity of making important decisions, Student, Hypochondriasis, Feelings of guilt, Work and interests and Energy.

The objective of this project is to carry out counterfactual simulations to obtain the possible factors that influence a certain individual to have suicidal ideation.

With it, health professionals who care for patients with suicidal ideation will be able to work with them more assertively, aiming to mitigate or even eliminate such ideation.

Steps for execution:

1) Upload the data.csv file by accessing the Files icon in the left corner menu, navigating to the folder containing the file. Click open and ignore the warning.

2) Access the Run environment/Run all menu

3) Enter data for each factor relating to the individual in question:
   
  	> Suicide in the family: Enter 1 for Yes and 0 for No.
   
  	> Capacity of making important decisions: Enter a number from 1 to 5, where: 1=I completely agree; 2=Agree; 3=Neutral; 4=Disagree;
       5=I completely disagree.
  
  	> Student: Enter a number from 0 to 1, where: 0=No; 1=Yes; 0.5=Part-time.
  
  	> Hypochondriasis: Enter a number from 0 to 4, where: 0=Absent; 1=Light; 2=Moderate; 3=Moderately severe; 4=Severe.
  
  	> Feelings of guilt: nter a number from 0 to 4, where: 0=Absent; 1=Doubtful or trivial: Self-censorship, feels that he has let
		people down; 2=Mild: Ideas of guilt or rumination about past mistakes or sinful acts; 3=Moderate: Current illness
		is a punishment. Delusions of guilt; 4=Severe: Hears accusatory or denunciatory voices and/or experiences threatening
		visual hallucinations.
  
  	> Work and interests: Enter a number from 0 to 4, where: 0=Absent: No difficulty; 1=Doubtful or trivial: Thoughts and feelings of
		incapacity, tiredness or weakness related to activities; work or hobbies; 2=Mild: Loss of interest in the activity;
		hobbies or work – reported directly by the patient, or indirectly in apathy, indecision and vacillation (feeling like you need to
		make an effort to work or carry out activities); 3=Moderate: Decrease in real time spent on activities or decrease in productivity;
		4=Serious: Stopped working due to current illness.

  	> Depression in the family: Enter 1 for Yes and 0 for No.
   
  	> History of alcohol use in the family: Enter 1 for Yes and 0 for No.
   
	  > Capacity of enjoying things: Enter a number from 1 to 5, where: 1=I completely agree; 2=Agree; 3=Neutral; 4=Disagree;
		5=I completely disagree.

	  > Drug use: Enter a number from 0 to 2, where: 0=None verified; 1=Abuse; 2=Dependent.
  
  	> Suicide: Enter a number from 0 to 4, where: : 0=Absent; 1=Feels that life is empty; 2=Death wishes; 3=Active suicidal thoughts;
		4=Serious suicide attempt.
  
  	> Anxiety: Enter a number from 0 to 4, where: 0=Absent: No difficulty; 1=Doubtful or trivial: Subjective tension
		and irritability; 2=Mild: Preoccupation with minor matters; 3=Moderate: Apprehensive attitude apparent on face or speech;
		4=Severe: Fears expressed without questioning.


4) After a few minutes, the value with the level of suicidal ideation predicted for the individual will be displayed

5) The last step is to carry out counterfactual** analyzes simulating changes in each of the factors. You can change each of the 10 factors and see if the level of suicidal ideation will be raised with the change.

Note: If you need to perform more simulations, run the last cell again by clicking on the icon icone.png

A change in the value of the level of suicidal ideation in a simulation may mean that the individual is sensitive to the
   factor in question.

	** Counterfactual thinking is a type of imaginative thinking characterized by the mental simulation of alternatives to past
	events with the aim of altering the sequence of events so that a different outcome is reached than the one that actually
	occurred. By carrying out simulations on the factors that presented themselves as possible causes of suicidal ideation,
	it is possible to conclude which factor would be important to work on as a priority in each individual.


In [1]:
#import libraries
import torch
import pandas as pd
import numpy as np

# Import dataset utils
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

import importlib
if importlib.util.find_spec('ipywidgets') is not None:
    from tqdm.auto import tqdm
else:
    from tqdm import tqdm

import networkx as nx

from typing import Union

In [2]:
dataframe = pd.read_csv('/content/data.csv', sep=';')

dataframe = dataframe.fillna(-1)
dataframe['Anos educacao formal'] = dataframe['Anos educacao formal'].replace(-5, -1)
dataframe['sexo'].replace({'M': 0, 'F': 1}, inplace=True)
df_suic = dataframe.copy()
df_suic = df_suic.astype(float)
df_suic['Chave'] = df_suic.index
df_suic.shape

(3953, 69)

In [3]:
notears_selected = [
   "Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    'Dep_familia',
    'Alc_familia',
    'Capaz de desfrutar das coisas',
    'Droga',
    'Suicidio',
    'Ansiedade'
     ]

# Read dataset

In [4]:
 df_novo = pd.DataFrame(columns = ["Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    'Dep_familia',
    'Alc_familia',
    'Capaz de desfrutar das coisas',
    'Droga',
    "Suicidio",
    "Ansiedade"])

# Reading features new individual

In [5]:
Suic_familia = int(input("Suicide in the family: Enter 1 for Yes and 0 for No.") )
Capaz_tomar_decisoes = int(input("Capacity of making important decisions: Enter a number from 1 to 5, with 1 being low capacity and 5 being high capacity.") )
Estudante = int(input("Student: Enter a number from 0 to 1, where: 0=No; 1=Yes; 0.5=Part-time.") )
Hipocondriase = int(input("Hypochondriasis: Enter a number from 0 to 4, with 0 being no and 4 being a high level.") )
Sentimento_culpa = int(input("Feelings of guilt: Enter a number from 0 to 4, with 0 being no and 4 being high.") )
Trabalho_interesses = int(input("Work and interests: Enter a number from 0 to 4, with 0 being no and 4 being high level.") )
Dep_familia = int(input("Depression in the family: Enter 1 for Yes and 0 for No.") )
Alc_familia = int(input("History of alcohol use in the family: Digite 1 para Sim e 0 para não.") )
Capaz_desfrutar = int(input("Capacity of enjoying things: nter a number from 1 to 5, where: 1=I completely agree; 2=Agree; 3=Neutral; 4=Disagree; 5=I completely disagree.") )
Droga = int(input("Drug use: Enter a number from 0 to 2, where: 0=None verified; 1=Abuse; 2=Dependent.") )
Suicidio = int(input("Suicide: Enter a number from 0 to 4, where 0 has no ideation and 4 has already made an attempt.") )
Ansiedade = int(input("Anxiety: Enter a number from 0 to 4, with 0 being no and 4 being high level.") )

Suicide in the family: Enter 1 for Yes and 0 for No.1
Capacity of making important decisions: Enter a number from 1 to 5, with 1 being low capacity and 5 being high capacity.0
Student: Enter a number from 0 to 1, where: 0=No; 1=Yes; 0.5=Part-time.3
Hypochondriasis: Enter a number from 0 to 4, with 0 being no and 4 being a high level.0
Feelings of guilt: Enter a number from 0 to 4, with 0 being no and 4 being high.2
Work and interests: Enter a number from 0 to 4, with 0 being no and 4 being high level.4
Depression in the family: Enter 1 for Yes and 0 for No.2
Histórico de uso de álcool na família: Digite 1 para Sim e 0 para não.1
Capacity of enjoying things: nter a number from 1 to 5, where: 1=I completely agree; 2=Agree; 3=Neutral; 4=Disagree; 5=I completely disagree.2
Drug use: Enter a number from 0 to 2, where: 0=None verified; 1=Abuse; 2=Dependent.1
Suicide: Enter a number from 0 to 4, where 0 has no ideation and 4 has already made an attempt.2
Anxiety: Enter a number from 0 to 4, w

In [6]:
df_novo=df_novo.append({'Suic_familia' : float(Suic_familia), 'Capaz de tomar decisões importantes' : float(Capaz_tomar_decisoes), 'Estudante' : float(Estudante),
                        'Hipocondriase' : float(Hipocondriase), 'Sentimentos_culpa' : float(Sentimento_culpa), 'Trabalho e interesses' : float(Trabalho_interesses),
                        'Dep_familia': float(Dep_familia), 'Alc_familia': float(Alc_familia),
                        'Capaz de desfrutar das coisas': float(Capaz_desfrutar), 'Droga' : float(Droga),
                        'Suicidio' : float(Suicidio), 'Ansiedade': float(Ansiedade)}, ignore_index = True)

<ipython-input-6-a02f1c562483>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_novo=df_novo.append({'Suic_familia' : float(Suic_familia), 'Capaz de tomar decisões importantes' : float(Capaz_tomar_decisoes), 'Estudante' : float(Estudante),


In [7]:
df_novo['Suic_familia'] = df_novo['Suic_familia'].astype(float)
df_novo['Capaz de tomar decisões importantes'] = df_novo['Capaz de tomar decisões importantes'].astype(float)
df_novo['Estudante'] = df_novo['Estudante'].astype(float)
df_novo['Hipocondriase'] = df_novo['Hipocondriase'].astype(float)
df_novo['Sentimentos_culpa'] = df_novo['Sentimentos_culpa'].astype(float)
df_novo['Trabalho e interesses'] = df_novo['Trabalho e interesses'].astype(float)
df_novo['Dep_familia'] = df_novo['Dep_familia'].astype(float)
df_novo['Alc_familia'] = df_novo['Alc_familia'].astype(float)
df_novo['Capaz de desfrutar das coisas'] = df_novo['Capaz de desfrutar das coisas'].astype(float)
df_novo['Droga'] = df_novo['Droga'].astype(float)
df_novo['Suicidio'] = df_novo['Suicidio'].astype(float)
df_novo['Ansiedade'] = df_novo['Ansiedade'].astype(float)

# Create causal datasets

In [8]:
# Dataset from X
class CausalDataset(Dataset):
    def __init__(self, X, target:Union[list, int]):
        self.X = torch.tensor(X, dtype=torch.double)
        self.target = target
        if isinstance(target, int):
            self.target = [target]


        # X is every variable except the target list
        dims = list(range(self.X.shape[1]))
        self.x = self.X[:, [i for i in dims if i not in self.target]]
        self.y = self.X[:, self.target]

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx].double(), self.y[idx].double()

In [9]:
class MLP(torch.nn.Module):
    def __init__(self, dim_list:list, add_dropout:bool=False):
        super(MLP, self).__init__()
        torch.manual_seed(3)
        self.layers = torch.nn.ModuleList()
        for i in range(len(dim_list) - 2):
            self.layers.append(torch.nn.Linear(dim_list[i], dim_list[i+1]))
            self.layers.append(torch.nn.ReLU())
            if add_dropout:
                self.layers.append(torch.nn.Dropout(0.3))

        self.layers.append(torch.nn.Linear(dim_list[-2], dim_list[-1]))
        self.double()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Out train pipeline

In [10]:
def trainPipeline(dataset, model, batch_size=64, train_split=0.8, verbose=True, max_epochs=100, patience=10, lr=0.001, weight_decay=0.0, loss_fn=nn.MSELoss()):
    train_size = int(train_split * len(dataset))
    test_size = len(dataset) - train_size
    torch.manual_seed(3)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    criterion = loss_fn
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    patience = patience

    train_losses = []
    test_losses = []
    for epoch in tqdm(range(max_epochs)):
        train_loss = 0
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        test_loss = 0
        model.eval()
        with torch.no_grad():
            for data, target in test_loader:
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item()
        test_loss /= len(test_loader)
        test_losses.append(test_loss)

        if verbose:
            print(f"Epoch {epoch} - Train loss: {train_loss} - Test loss: {test_loss}")

        # add early stopping with patience variable
        if epoch > 10:
            all_patience = test_losses[-patience:]
            test_loss = test_losses[-1]
            if all([test_loss > x for x in all_patience]):
                print("Early stopping")
                break

    # test model on test set and calculate accuracy
    model.eval()
    with torch.no_grad():
        y_pred_test = []
        y_true_test = []
        y_pred_train = []
        y_true_train = []

        for data, target in train_loader:
            output = model(data)
            y_pred_train.append(output)
            y_true_train.append(target)
        for data, target in test_loader:
            output = model(data)
            y_pred_test.append(output)
            y_true_test.append(target)

        y_pred_test = torch.cat(y_pred_test, dim=0)
        y_true_test = torch.cat(y_true_test, dim=0)
        y_pred_train = torch.cat(y_pred_train, dim=0)
        y_true_train = torch.cat(y_true_train, dim=0)

        train_error = torch.abs(y_pred_train - y_true_train)
        test_error = torch.abs(y_pred_test - y_true_test)

        # calculate accuracy for each target
        test_accuracy = []
        train_accuracy = []
        for i in range(len(y_true_test[0])):
            test_accuracy.append(torch.sum(test_error[:, i] < 0.5) / len(test_error))
            train_accuracy.append(torch.sum(train_error[:, i] < 0.5) / len(train_error))

        print(f"Final Train accuracy: {train_accuracy}")
        print(f"Final Test accuracy: {test_accuracy}")

    metrics = {
        "train_loss": train_losses,
        "test_loss": test_losses,
        "train_error": train_error,
        "test_error": test_error,
        "test_accuracy": test_accuracy
    }

    return model, metrics


## First train one model for each target and get errors value

## Now add the errors to the dataset and train the counterfactual model

In [11]:
# Create the counterfactual model

class CounterfactualModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CounterfactualModel, self).__init__()
        self.individuality_model = MLP([input_size, hidden_size, hidden_size, hidden_size, 1], add_dropout=True)
        self.counterfactual_model = MLP([input_size+1, hidden_size, hidden_size, hidden_size, output_size], add_dropout=True)

    def individuality(self, x):
        return self.individuality_model(x)

    def counterfactual(self, x, i):
        x = torch.cat([x, i], dim=1)
        return self.counterfactual_model(x)

    def forward(self, x):
        i = self.individuality(x)
        y = self.counterfactual(x, i)
        return y

# Now we are going to redo everything, but with the NOTEAR features

In [12]:
df_suic_notears = df_suic.copy()
df_suic_notears = df_suic_notears[notears_selected]
df_suic_notears=pd.concat([df_suic_notears,df_novo], ignore_index=True)

# Create models for each target
targets = ["Suicidio", "Ansiedade"]
targets_idx = [df_suic_notears.columns.get_loc(c) for c in targets]

# Create the dataset
notears_suic_dataset = CausalDataset(df_suic_notears.values, targets_idx[0])
notears_anx_dataset = CausalDataset(df_suic_notears.values, targets_idx[1])
sample_x, sample_y = notears_suic_dataset[0]

# Now we predict the targets with a MLP
suic_causal_model = MLP([sample_x.shape[0], 64, 64, 64, 1], add_dropout=True)
anx_causal_model = MLP([sample_x.shape[0], 64, 64, 64, 1], add_dropout=True)

suic_causal_model, suic_causal_metrics = trainPipeline(notears_suic_dataset, suic_causal_model, verbose=False, max_epochs=1000, weight_decay=1e-5)
anx_causal_model, anx_causal_metrics = trainPipeline(notears_anx_dataset, anx_causal_model, verbose=False, max_epochs=1000, weight_decay=1e-5)



  0%|          | 0/1000 [00:00<?, ?it/s]

Final Train accuracy: [tensor(0.5390)]
Final Test accuracy: [tensor(0.3906)]


  0%|          | 0/1000 [00:00<?, ?it/s]

Final Train accuracy: [tensor(0.5378)]
Final Test accuracy: [tensor(0.4690)]


In [13]:
# calculate individuality
suic_causal_erros = []
anx_causal_erros = []
with torch.no_grad():
    for data, y_true in notears_suic_dataset:
        y_pred = suic_causal_model(data)
        error = torch.abs(y_pred - y_true).item()
        suic_causal_erros.append(error)
    for data, y_true in notears_anx_dataset:
        y_pred = anx_causal_model(data)
        error = torch.abs(y_pred - y_true).item()
        anx_causal_erros.append(error)

suic_causal_erros = np.array(suic_causal_erros)
anx_causal_erros = np.array(anx_causal_erros)

# add individuality to the dataframe
df_suic_notears_with_errors = df_suic_notears.copy()
df_suic_notears_with_errors["Suicidio_error"] = suic_causal_erros
df_suic_notears_with_errors["Ansiedade_error"] = anx_causal_erros

# create the causal dataset with the individuality
targets_idx = [df_suic_notears_with_errors.columns.get_loc(c) for c in targets]
notears_causal_dataset_error = CausalDataset(df_suic_notears_with_errors.values, targets_idx)
sample_x, sample_y = notears_causal_dataset_error[0]

# create the counterfactual model
causal_cf_model = CounterfactualModel(sample_x.shape[0], 64, sample_y.shape[0])

# train the counterfactual model
causal_cf_model, causal_cf_metrics = trainPipeline(notears_causal_dataset_error, causal_cf_model, verbose=False, max_epochs=1000, weight_decay=1e-5)


  0%|          | 0/1000 [00:00<?, ?it/s]

Final Train accuracy: [tensor(0.8188), tensor(0.7607)]
Final Test accuracy: [tensor(0.7193), tensor(0.6839)]


In [14]:
# Load the dataset
targets = ["Suicidio", "Ansiedade"]
df = df_suic_notears_with_errors

# load models
counterfact = causal_cf_model
counterfact = counterfact.eval()

# Initial prediction

In [15]:
def contrafactual(df):
  df_suic_notears_with_errors = df
  # Select two individuals
  dataset = CausalDataset(df_suic_notears_with_errors.values, [df.columns.get_loc(c) for c in targets])

  idx = [0,3953]

  data1, y1 = dataset[idx[0]]
  data2, y2 = dataset[idx[1]]

  # predict the counterfactuals for crossed individuals
  with torch.no_grad():
      i1 = counterfact.individuality(data1.unsqueeze(0))
      i2 = counterfact.individuality(data2.unsqueeze(0))

      cf11 = counterfact.counterfactual(data1.unsqueeze(0), i1)
      cf12 = counterfact.counterfactual(data2.unsqueeze(0), i2)

      print(f"Counterfactual Suicide: {cf12.squeeze(0).round().numpy().tolist()[0]}")
      print()

In [16]:
print('Initial prediction')
contrafactual(df_suic_notears_with_errors)

Initial prediction
Counterfactual Suicide: 3.0



# Counterfactual simulations

In [17]:
print('Now to carry out the simulations, enter the value indicated according to the feature you want to modify')
print('1 - Suicide in the family')
print('2 - Capacity of making important decisions')
print('3 - Student')
print('4 - Hypochondriasis')
print('5 - Feelings of guilt')
print('6 - Work and interests')
print('7 - Depression in the family')
print('8 - History of alcohol use in the family')
print('9 - Capacity of enjoying things')
print('10 - Drug use')

Now to carry out the simulations, enter the value indicated according to the feature you want to modify
1 - Suicide in the family
2 - Capacity of making important decisions
3 - Student
4 - Hypochondriasis
5 - Feelings of guilt
6 - Work and interests
7 - Depression in the family
8 - History of alcohol use in the family
9 - Capacity of enjoying things
10 - Drug use


In [21]:
def alter_feature(df,Variavel,valor):
  df_suic_notears_with_errors = df
  if Variavel == '1':
    df_suic_notears_with_errors['Suic_familia'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Suicide in the family')
  elif Variavel == '2':
    df_suic_notears_with_errors['Capaz de tomar decisões importantes'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Capacity of making important decisions')
  elif Variavel == '3':
    df_suic_notears_with_errors['Estudante'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Student')
  elif Variavel == '4':
    df_suic_notears_with_errors['Hipocondriase'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Hipocondriase')
  elif Variavel == '5':
    df_suic_notears_with_errors['Sentimentos_culpa'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Feelings of guilt')
  elif Variavel == '6':
    df_suic_notears_with_errors['Trabalho e interesses'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Work and interests')
  elif Variavel == '7':
    df_suic_notears_with_errors['Dep_familia'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Depression in the family')
  elif Variavel == '8':
    df_suic_notears_with_errors['Alc_familia'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('History of alcohol use in the family')
  elif Variavel == '9':
    df_suic_notears_with_errors['Capaz de desfrutar das coisas'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Capacity of enjoying things')
  else:
    df_suic_notears_with_errors['Droga'].loc[(df_suic_notears_with_errors.index==3953)] = float(Valor)
    print('Drug use')
  return df_suic_notears_with_errors

In [22]:
for i in range(1,10):
  df_simula = df_suic_notears_with_errors.copy()
  print('Enter the value indicated according to the feature you want to modify')
  Variavel = input("1 Suicide in the family, 2 Capacity of making important decisions, 3 Student, 4 Hypochondriasis, 5 Feelings of guilt, 6 Work and interests, 7 Depression in the family, 8 Alcohol use in the family, 9 Capacity of enjoying things and 10 Drug use")
  Valor = input("Enter the new value")
  df_simula = alter_feature(df_simula, Variavel, Valor)
  contrafactual(df_simula)
  if i<10:
    simulacao = input("Do you want to run a new simulation? Enter 1 for Yes and 0 for No")
    if simulacao=='0':
      break

Enter the value indicated according to the feature you want to modify
1 Suicide in the family, 2 Capacity of making important decisions, 3 Student, 4 Hypochondriasis, 5 Feelings of guilt, 6 Work and interests, 7 Depression in the family, 8 Alcohol use in the family, 9 Capacity of enjoying things and 10 Drug use1
Enter the new value0
Suicide in the family
Counterfactual Suicide: 2.0

Do you want to run a new simulation? Enter 1 for Yes and 0 for No1
Enter the value indicated according to the feature you want to modify
1 Suicide in the family, 2 Capacity of making important decisions, 3 Student, 4 Hypochondriasis, 5 Feelings of guilt, 6 Work and interests, 7 Depression in the family, 8 Alcohol use in the family, 9 Capacity of enjoying things and 10 Drug use2
Enter the new value1
Capacity of making important decisions
Counterfactual Suicide: 3.0

Do you want to run a new simulation? Enter 1 for Yes and 0 for No0
